In [20]:
import pandas as pd
import numpy as np
import googlemaps

gmaps_key = googlemaps.Client(key='')

pd.set_option('display.max_columns', None)

In [45]:
# import csv and filter to necessary columns
df_addr = pd.read_csv('../Ames Real Estate Data.csv', low_memory=False)
df_addr = df_addr.loc[:,['MapRefNo', 'Prop_Addr']]
df_addr.drop_duplicates(inplace=True)

(22213, 2)
(21898, 2)


1240 TRUMAN PL          2
4300 MARICOPA DR        2
3209 MAPLEWOOD CIR      2
1711 TOP O HOLLOW RD    2
1210 ORCHARD DR         2
                       ..
1816 MEADOWLANE AVE     1
1822 MEADOWLANE AVE     1
1904 MEADOWLANE AVE     1
1910 MEADOWLANE AVE     1
1820 S DAYTON PL        1
Name: Prop_Addr, Length: 21864, dtype: int64

In [47]:
# merge with House Price dataset
merge1 = pd.read_csv('../Ames_HousePrice.csv', index_col=0)
df = merge1.merge(df_addr, how='left', left_on = 'PID', right_on = 'MapRefNo')

(2580, 81)
(2580, 83)


In [48]:
# check for NaN in address column
df[df.Prop_Addr == None]

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,MapRefNo,Prop_Addr


In [49]:
# re-format address to include Ames, Iowa, USA
df['Prop_Addr'] = df['Prop_Addr'].map(lambda x: f'{x}, AMES, IOWA, USA', na_action='ignore')

In [50]:
# test API with one address
add_1 = df['Prop_Addr'][0]
g = gmaps_key.geocode(add_1)
lat = g[0]["geometry"]["location"]["lat"]
lon = g[0]["geometry"]["location"]["lng"]
print(str(lat) + ', ' + str(lon))

42.0178137, -93.65142759999999


In [51]:
# create function to apply to entire series of addresses
def geocode(x):
    try:
        g = gmaps_key.geocode(x)
        lat = g[0]["geometry"]["location"]["lat"]
        lng = g[0]["geometry"]["location"]["lng"]
        return (lat, lng)
    except:
        return None

In [52]:
df['geocode'] = df['Prop_Addr'].apply(geocode)

In [56]:
# parse out Lat and Lng into new columns
df['lat'] = df['geocode'].map(lambda x: None if x == None else x[0])
df['lng'] = df['geocode'].map(lambda x: None if x == None else x[1])

In [63]:
df.head()

,PID,GrLivArea,SalePrice,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,MapRefNo,Prop_Addr,geocode,lat,lng
0,909176150,856,126000,30,RL,NaN,7890,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Norm,Norm,1Fam,1Story,6,6,1939,1950,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,238.0,Unf,0.0,618.0,856.0,GasA,TA,Y,SBrkr,856,0,0,1.0,0.0,1,0,2,1,TA,4,Typ,1,Gd,Detchd,1939.0,Unf,2.0,399.0,TA,TA,Y,0,0,0,0,166,0,NaN,NaN,NaN,0,3,2010,WD,Normal,909176150.0,"436 HAYWARD AVE, AMES, IOWA, USA","(42.0178137, -93.65142759999999)",42.017814,-93.651428
1,905476230,1049,139500,120,RL,42.0,4235,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,TwnhsE,1Story,5,5,1984,1984,Gable,CompShg,HdBoard,HdBoard,BrkFace,149.0,Gd,TA,CBlock,Gd,TA,Mn,GLQ,552.0,ALQ,393.0,104.0,1049.0,GasA,TA,Y,SBrkr,1049,0,0,1.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1984.0,Fin,1.0,266.0,TA,TA,Y,0,105,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,905476230.0,"3416 WEST ST, AMES, IOWA, USA","(42.0247087, -93.6641683)",42.024709,-93.664168
2,911128020,1001,124900,30,C (all),60.0,6060,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,1Fam,1Story,5,9,1930,2007,Hip,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,BrkTil,TA,TA,No,ALQ,737.0,Unf,0.0,100.0,837.0,GasA,Ex,Y,SBrkr,1001,0,0,0.0,0.0,1,0,2,1,Gd,5,Typ,0,NaN,Detchd,1930.0,Unf,1.0,216.0,TA,Po,N,154,0,42,86,0,0,NaN,NaN,NaN,0,11,2007,WD,Normal,911128020.0,"320 S 2ND ST, AMES, IOWA, USA","(42.0214065, -93.614914)",42.021406,-93.614914
3,535377150,1039,114000,70,RL,80.0,8146,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,2Story,4,8,1900,2003,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,Gd,BrkTil,Fa,TA,No,Unf,0.0,Unf,0.0,405.0,405.0,GasA,Gd,Y,SBrkr,717,322,0,0.0,0.0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1940.0,Unf,1.0,281.0,TA,TA,N,0,0,168,0,111,0,NaN,NaN,NaN,0,5,2009,WD,Normal,535377150.0,"1524 DOUGLAS AVE, AMES, IOWA, USA","(42.0380884, -93.6120847)",42.038088,-93.612085
4,534177230,1665,227000,60,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,8,6,2001,2001,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,643.0,Unf,0.0,167.0,810.0,GasA,Ex,Y,SBrkr,810,855,0,1.0,0.0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2001.0,Fin,2.0,528.0,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,11,2009,WD,Normal,534177230.0,"2304 FILLMORE AVE, AMES, IOWA, USA","(42.0449, -93.6318933)",42.044900,-93.631893


In [64]:
# write lat and lng back to csv
df.to_csv('Ames_House_Price_w_geo.csv')